In [1]:
import os
import dotenv
import json
import inspect
import types
import torch

from pydantic import BaseModel
from langchain.agents import tool
from openai import OpenAI
from openai.types.chat.chat_completion import ChatCompletion
from sentence_transformers import SentenceTransformer, util
from langchain_core.utils.function_calling import convert_to_openai_tool

/home/saul/.pyenv/versions/3.11.8/envs/langchain/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Define Tools

OpenAI knows  how to recognize when to use tool functions and structure the input for them. However the OpenAI API takes JSON as input and we should convert our functions to JSON.

Converting a Python function to JSON directly isn't straightforward because JSON is a data interchange format, and functions in Python are executable code. However, you can represent certain aspects of a function in JSON, such as its name, arguments, and possibly its source code.

In [2]:
def add_two_numbers(a: float, b: float) -> float:
    """This function will add a and b together and return the result."""
    return a + b

def multiply_two_numbers(a: float, b: float) -> float:
    """This function will multiply a by b and return the result."""
    return a * b

def divide_two_numbers(a: float, b: float) -> float:
    """This function will divide a by b and return the result."""
    return a / b

### Manual JSON Conversion

In [3]:
add_two_numbers_json = {
    'type': 'function',
    'function': {
        'name': 'add_two_numbers',
        'description': 'add_two_numbers(a: float, b: float) -> float - This function will add a and b together and return the result.',
        'parameters': {
            'title': 'add_two_numbersSchemaSchema',
            'type': 'object',
            'properties': {
                'a': {'title': 'A', 'type': 'number'},
                'b': {'title': 'B', 'type': 'number'}
                },
            'required': ['a', 'b']
            }
        }
    }

add_two_numbers_json

{'type': 'function',
 'function': {'name': 'add_two_numbers',
  'description': 'add_two_numbers(a: float, b: float) -> float - This function will add a and b together and return the result.',
  'parameters': {'title': 'add_two_numbersSchemaSchema',
   'type': 'object',
   'properties': {'a': {'title': 'A', 'type': 'number'},
    'b': {'title': 'B', 'type': 'number'}},
   'required': ['a', 'b']}}}

### Function to JSON with langchain

Langchain generally does pretty well but messes up on the default arguments.

In [4]:
function_calling_def = convert_to_openai_tool(tool(add_two_numbers))
function_calling_def

{'type': 'function',
 'function': {'name': 'add_two_numbers',
  'description': 'add_two_numbers(a: float, b: float) -> float - This function will add a and b together and return the result.',
  'parameters': {'type': 'object',
   'properties': {'a': {'type': 'number'}, 'b': {'type': 'number'}},
   'required': ['a', 'b']}}}

### Function to JSON with Pydantic

Pydantic will give more control over the conversion process. 

However, it requires more manual work. Notice that Pydantic is missing the description of the function. We can add it manually.

In [5]:
class AddTwoNumbers(BaseModel):
    a: float
    b: float

AddTwoNumbers.model_json_schema()

{'properties': {'a': {'title': 'A', 'type': 'number'},
  'b': {'title': 'B', 'type': 'number'}},
 'required': ['a', 'b'],
 'title': 'AddTwoNumbers',
 'type': 'object'}

# Application

In [6]:
dotenv.load_dotenv()
client = OpenAI(api_key = os.environ.get("OPENAI_API_KEY"))
messages = [
    {
        "role": "system",
        "content": "You are a helpful assistant that can perform math operations. Please help me with the following math problem.",
        },
    {
        "role": "user", 
        "content": "Please add 1 and 5",
        },
    ]
function = convert_to_openai_tool(tool(add_two_numbers))

In [7]:
response = client.chat.completions.create(
    model="gpt-3.5-turbo-16k-0613", 
    messages=messages, 
    tools=[function],
    max_tokens=100,
    temperature=0.0,
    )

We print out the result,

Notice that result.choices.[0].message.content is None.

We see that the reason the call ended was due to 'tool_calls', we can figure out which tool to call and finish the call and the arguments to pass.

In [8]:
response

"""
ChatCompletion(
    id='chatcmpl-9FpiN2BywNG8jHVgaRTkEfMRgugGW', 
    choices=[
        Choice(
            finish_reason='tool_calls', 
            index=0, 
            logprobs=None, 
            message=ChatCompletionMessage(
                content=None, 
                role='assistant', 
                function_call=None, 
                tool_calls=[
                    ChatCompletionMessageToolCall(
                        id='call_rUH4nMcpv9syrXz0SBJnk0oW', 
                        function=Function(
                            arguments='{\n  "a": 1,\n  "b": 5\n}', 
                            name='add_two_numbers',
                            ), 
                        type='function',
                        )
                    ]
                )
            )
        ], 
    created=1713560739, 
    model='gpt-3.5-turbo-16k-0613', 
    object='chat.completion', 
    system_fingerprint=None, 
    usage=CompletionUsage(
        completion_tokens=23, 
        prompt_tokens=98, 
        total_tokens=121,
        )
    )
"""

'\nChatCompletion(\n    id=\'chatcmpl-9FpiN2BywNG8jHVgaRTkEfMRgugGW\', \n    choices=[\n        Choice(\n            finish_reason=\'tool_calls\', \n            index=0, \n            logprobs=None, \n            message=ChatCompletionMessage(\n                content=None, \n                role=\'assistant\', \n                function_call=None, \n                tool_calls=[\n                    ChatCompletionMessageToolCall(\n                        id=\'call_rUH4nMcpv9syrXz0SBJnk0oW\', \n                        function=Function(\n                            arguments=\'{\n  "a": 1,\n  "b": 5\n}\', \n                            name=\'add_two_numbers\',\n                            ), \n                        type=\'function\',\n                        )\n                    ]\n                )\n            )\n        ], \n    created=1713560739, \n    model=\'gpt-3.5-turbo-16k-0613\', \n    object=\'chat.completion\', \n    system_fingerprint=None, \n    usage=CompletionUsage(

In [9]:
response.choices[0].message.tool_calls[0].function.name

'add_two_numbers'

# Well how do I pull the information from the result?

In [10]:
def gpt_process_function_calling(gpt_response: ChatCompletion):
    # Check to see if the call terminated on a function call.
    finish_reason = gpt_response.choices[0].finish_reason
    # We check if we finished for an explicit function call or if we finished because of a long query
    # and gpt suggests a function call
    if finish_reason == "tool_calls":
        function_name = gpt_response.choices[0].message.tool_calls[0].function.name
        arguments = json.loads(gpt_response.choices[0].message.tool_calls[0].function.arguments)
        func = globals()[function_name]
        return func(**arguments)
    else:
        # if not just pass the response through.
        return gpt_response.choices[0].message.content

In [11]:
answer = gpt_process_function_calling(response)
answer

6

# How do I call all functions?

In [12]:
funcs = ["add_two_numbers", "multiply_two_numbers", "divide_two_numbers"]
functions = [convert_to_openai_tool(tool(globals()[t])) for t in funcs]

In [13]:
functions

[{'type': 'function',
  'function': {'name': 'add_two_numbers',
   'description': 'add_two_numbers(a: float, b: float) -> float - This function will add a and b together and return the result.',
   'parameters': {'type': 'object',
    'properties': {'a': {'type': 'number'}, 'b': {'type': 'number'}},
    'required': ['a', 'b']}}},
 {'type': 'function',
  'function': {'name': 'multiply_two_numbers',
   'description': 'multiply_two_numbers(a: float, b: float) -> float - This function will multiply a by b and return the result.',
   'parameters': {'type': 'object',
    'properties': {'a': {'type': 'number'}, 'b': {'type': 'number'}},
    'required': ['a', 'b']}}},
 {'type': 'function',
  'function': {'name': 'divide_two_numbers',
   'description': 'divide_two_numbers(a: float, b: float) -> float - This function will divide a by b and return the result.',
   'parameters': {'type': 'object',
    'properties': {'a': {'type': 'number'}, 'b': {'type': 'number'}},
    'required': ['a', 'b']}}}]

In [14]:
workflow = [
	"add 1 and 5",
	"multiply 5 by the number: ",
	"divide the following number up by 15: "
	"Repeat the word 'duck' this many times please: "
]

* Notes for the future.

ChatGPT seems to have trouble with the mutiplication line. It seems to just be copying the arguments from the previous line; a = 1, b = 5 instead of a = 6, b = 5.

Another thing is that it's not repeating the word duck. This is most likely due to the 

In [15]:
outputs = []
raw_output = ""
for instruction in workflow:
    messages = [
        {
            "role": "system",
            "content": "",
            },
        {
            "role": "user",
            "content": instruction + f"{raw_output}",
            }
        ]
    print(messages)
    raw_output = client.chat.completions.create(
        model="gpt-3.5-turbo-16k-0613", 
        messages=messages, 
        tools=functions,
        max_tokens=100,
        temperature=0.0,
        )
    print(raw_output)
    output = gpt_process_function_calling(raw_output)
    print(output)

    outputs.append(output)

[{'role': 'system', 'content': ''}, {'role': 'user', 'content': 'add 1 and 5'}]
ChatCompletion(id='chatcmpl-9Fr4OQzCbXgX2nJfzKguUlKibUNaK', choices=[Choice(finish_reason='tool_calls', index=0, logprobs=None, message=ChatCompletionMessage(content=None, role='assistant', function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_p9xV6U0Yse1C2L06DymISY4x', function=Function(arguments='{\n  "a": 1,\n  "b": 5\n}', name='add_two_numbers'), type='function')]))], created=1713565948, model='gpt-3.5-turbo-16k-0613', object='chat.completion', system_fingerprint=None, usage=CompletionUsage(completion_tokens=23, prompt_tokens=171, total_tokens=194))
6
[{'role': 'system', 'content': ''}, {'role': 'user', 'content': 'multiply 5 by the number: ChatCompletion(id=\'chatcmpl-9Fr4OQzCbXgX2nJfzKguUlKibUNaK\', choices=[Choice(finish_reason=\'tool_calls\', index=0, logprobs=None, message=ChatCompletionMessage(content=None, role=\'assistant\', function_call=None, tool_calls=[ChatCompletionMessageT

In [16]:
outputs

[6, 5, 3.0]

### Function finding

In [17]:
# Assign the closest two tools for each step in the workflow
embedder = SentenceTransformer('all-MiniLM-L6-v2')

# List of all functions
function_registry = ["add_two_numbers", "multiply_two_numbers", "divide_two_numbers"]

# Use the inspect library to get a string version of the function
# including the docstring
function_descriptions = [inspect.getsource(globals()[func]) for func in function_registry]

# Embed every function
function_embeddings = embedder.encode(function_descriptions, convert_to_tensor=True)

top_k = min(1, len(function_descriptions))
workflow_functions = []

# step through each workflow instruction and encode it.
for query in workflow:
    query_embedding = embedder.encode(query, convert_to_tensor=True)

    # We use cosine-similarity and torch.topk to find the highest 5 scores
    cos_scores = util.cos_sim(query_embedding, function_embeddings)[0]
    top_results = torch.topk(cos_scores, k=top_k)
	# We only add a function if it's closer than .25 by cosine distance.
    if max(cos_scores) > .2:
        workflow_functions.append([function_registry[i] for i in top_results.indices.tolist()])
    else:
        workflow_functions.append([])
workflow_functions

[['add_two_numbers'], ['multiply_two_numbers'], ['divide_two_numbers']]

In [18]:
top_results

torch.return_types.topk(
values=tensor([0.3775], device='cuda:0'),
indices=tensor([2], device='cuda:0'))

In [19]:
outputs = []
output = ""
for instruction, functions in zip(workflow, workflow_functions):
    kwargs = {}
    if len(functions) > 0:
        functions = [convert_to_openai_tool(tool(globals()[t])) for t in functions]
        kwargs = {"tools": functions}
    messages = [{"role": "system", "content": ""},
                {"role": "user", "content": instruction + f" {output}"}]
    print(messages)
    output = client.chat.completions.create(
        model="gpt-3.5-turbo-16k-0613", 
        messages=messages,
        **kwargs,
        max_tokens=100,
        temperature=0.0,
        )
    output = gpt_process_function_calling(output)
    outputs.append(output)

[{'role': 'system', 'content': ''}, {'role': 'user', 'content': 'add 1 and 5 '}]
[{'role': 'system', 'content': ''}, {'role': 'user', 'content': 'multiply 5 by the number:  6'}]
[{'role': 'system', 'content': ''}, {'role': 'user', 'content': "divide the following number up by 15: Repeat the word 'duck' this many times please:  30"}]


In [20]:
outputs

[6, 30, 2.0]

### Just messing around for chat

In [26]:
messages = [{"role": "system", "content": ""},
            {"role": "user", "content": "repeat the word duck 20 times"}]

In [21]:
output = client.chat.completions.create(
    model="gpt-3.5-turbo-16k-0613", 
    messages=messages,
    max_tokens=100,
    temperature=0.0,
    )

In [30]:
output.choices[0].message.content

'duck duck duck duck duck duck duck duck duck duck duck duck duck duck duck duck duck duck duck'

In [27]:
messages.append({"role": "assistant", "content": output.choices[0].message.content})
messages.append({"role": "user", "content": "Now tell me why did the chicken cross the road?"})

In [28]:
messages

[{'role': 'system', 'content': ''},
 {'role': 'user', 'content': 'repeat the word duck 20 times'},
 {'role': 'assistant',
  'content': 'duck duck duck duck duck duck duck duck duck duck duck duck duck duck duck duck duck duck duck'},
 {'role': 'user',
  'content': 'Now tell me why did the chicken cross the road?'}]

In [29]:
output2 = client.chat.completions.create(
    model="gpt-3.5-turbo-16k-0613", 
    messages=messages,
    max_tokens=100,
    temperature=0.0,
    )

In [31]:
output2.choices[0].message.content

'The classic answer to why the chicken crossed the road is to get to the other side. However, this answer is often used as a humorous or ironic response to a seemingly simple question. It\'s a play on words, as "the other side" can refer to the other side of the road, but it can also be interpreted as crossing over to the afterlife. Ultimately, the question is meant to be a lighthearted joke rather than a serious inquiry.'